In [3]:
import pandas as pd
import re

def normalize_pgc(pgc_value):
    """
    Normalise les valeurs PGC en extrayant le nombre et en supprimant les zéros de tête.
    Ex: 'PGC045875' -> '45875', 'PGC45875' -> '45875'
    """
    if pd.isna(pgc_value):
        return None
    
    # Convertir en string au cas où
    pgc_str = str(pgc_value).strip()
    
    # Extraire les chiffres après 'PGC'
    match = re.match(r'PGC(\d+)', pgc_str, re.IGNORECASE)
    if match:
        # Supprimer les zéros de tête et retourner le nombre
        return str(int(match.group(1)))
    
    # Si le format ne correspond pas, retourner la valeur originale
    return pgc_str

def remove_pgc_duplicates(csv_file_path, output_file_path=None):
    """
    Supprime les doublons basés sur la colonne PGC normalisée.
    Garde la ligne avec le nom le plus long parmi les doublons.
    Préserve les lignes sans valeur PGC.
    
    Args:
        csv_file_path (str): Chemin vers le fichier CSV d'entrée
        output_file_path (str): Chemin vers le fichier CSV de sortie (optionnel)
    
    Returns:
        pandas.DataFrame: DataFrame sans doublons
    """
    
    # Lire le fichier CSV
    print(f"Lecture du fichier: {csv_file_path}")
    df = pd.read_csv(csv_file_path)
    
    print(f"Nombre de lignes avant suppression des doublons: {len(df)}")
    
    # Vérifier que les colonnes PGC et Name existent
    if 'PGC' not in df.columns:
        raise ValueError("La colonne 'PGC' n'existe pas dans le fichier CSV")
    if 'Name' not in df.columns:
        raise ValueError("La colonne 'Name' n'existe pas dans le fichier CSV")
    
    # Séparer les lignes avec et sans valeur PGC
    df_with_pgc = df[df['PGC'].notna() & (df['PGC'] != '')].copy()
    df_without_pgc = df[df['PGC'].isna() | (df['PGC'] == '')].copy()
    
    print(f"Lignes avec PGC: {len(df_with_pgc)}")
    print(f"Lignes sans PGC (préservées): {len(df_without_pgc)}")
    
    if len(df_with_pgc) == 0:
        print("Aucune ligne avec valeur PGC à traiter")
        return df
    
    # Créer une colonne temporaire avec les valeurs PGC normalisées
    df_with_pgc['PGC_normalized'] = df_with_pgc['PGC'].apply(normalize_pgc)
    
    # Créer une colonne avec la longueur du nom pour le tri
    df_with_pgc['Name_length'] = df_with_pgc['Name'].astype(str).str.len()
    
    # Afficher quelques exemples de normalisation
    print("\nExemples de normalisation:")
    sample_pgc = df_with_pgc[['PGC', 'PGC_normalized']].drop_duplicates().head(10)
    for _, row in sample_pgc.iterrows():
        print(f"  {row['PGC']} -> {row['PGC_normalized']}")
    
    # Trier par PGC_normalized et longueur de nom (décroissant pour garder le plus long)
    df_sorted = df_with_pgc.sort_values(['PGC_normalized', 'Name_length'], 
                                       ascending=[True, False])
    
    # Supprimer les doublons en gardant la première occurrence (nom le plus long)
    df_dedup_with_pgc = df_sorted.drop_duplicates(subset=['PGC_normalized'], keep='first')
    
    # Supprimer les colonnes temporaires
    df_dedup_with_pgc = df_dedup_with_pgc.drop(['PGC_normalized', 'Name_length'], axis=1)
    
    # Combiner les lignes sans PGC avec les lignes dédupliquées
    df_final = pd.concat([df_dedup_with_pgc, df_without_pgc], ignore_index=True)
    
    print(f"Nombre de lignes après suppression des doublons: {len(df_final)}")
    print(f"Nombre de doublons supprimés: {len(df_with_pgc) - len(df_dedup_with_pgc)}")
    
    # Sauvegarder si un chemin de sortie est spécifié
    if output_file_path:
        df_final.to_csv(output_file_path, index=False)
        print(f"Fichier sauvegardé: {output_file_path}")
    
    return df_final

# Fonction pour identifier les doublons avant suppression (optionnel, pour vérification)
def find_pgc_duplicates(csv_file_path):
    """
    Identifie et affiche les groupes de doublons pour vérification.
    Montre aussi la longueur des noms pour voir lequel sera conservé.
    """
    df = pd.read_csv(csv_file_path)
    
    # Filtrer seulement les lignes avec PGC
    df_with_pgc = df[df['PGC'].notna() & (df['PGC'] != '')].copy()
    
    if len(df_with_pgc) == 0:
        print("Aucune ligne avec valeur PGC à analyser")
        return
    
    df_with_pgc['PGC_normalized'] = df_with_pgc['PGC'].apply(normalize_pgc)
    
    # Trouver les valeurs normalisées qui apparaissent plus d'une fois
    duplicated_pgc = df_with_pgc[df_with_pgc.duplicated(subset=['PGC_normalized'], keep=False)]
    
    if len(duplicated_pgc) > 0:
        print("Groupes de doublons détectés:")
        for pgc_norm in duplicated_pgc['PGC_normalized'].unique():
            group = duplicated_pgc[duplicated_pgc['PGC_normalized'] == pgc_norm]
            print(f"\nGroupe PGC {pgc_norm}:")
            # Trier par longueur de nom pour montrer lequel sera gardé
            group_sorted = group.sort_values('Name', key=lambda x: x.str.len(), ascending=False)
            for i, (_, row) in enumerate(group_sorted.iterrows()):
                marker = " [GARDE]" if i == 0 else ""
                Name_len = len(str(row['Name'])) if pd.notna(row['Name']) else 0
                print(f"  - {row['PGC']} | nom: '{row['Name']}' (longueur: {Name_len}){marker}")
    else:
        print("Aucun doublon détecté")

# Exemple d'utilisation
if __name__ == "__main__":
    # Remplacez par le chemin de votre fichier CSV
    input_file = "galaxies_fusionnees.csv"
    output_file = "../data/new_data.csv"
    
    try:
        # Optionnel: Identifier les doublons avant suppression
        print("=== Identification des doublons ===")
        find_pgc_duplicates(input_file)
        
        print("\n=== Suppression des doublons ===")
        # Supprimer les doublons
        df_clean = remove_pgc_duplicates(input_file, output_file)
        
        print("\nTraitement terminé avec succès!")
        
    except FileNotFoundError:
        print(f"Erreur: Le fichier '{input_file}' n'existe pas.")
        print("Veuillez modifier la variable 'input_file' avec le bon chemin.")
    except Exception as e:
        print(f"Erreur: {e}")

=== Identification des doublons ===
Groupes de doublons détectés:

Groupe PGC 46957:
  - PGC46957 | nom: 'NGC5128' (longueur: 7) [GARDE]
  - PGC046957 | nom: 'CenA' (longueur: 4)

Groupe PGC 45916:
  - PGC045916 | nom: 'ESO269-037' (longueur: 10) [GARDE]
  - PGC45916 | nom: 'ESO269-066' (longueur: 10)
  - PGC045916 | nom: 'ESO269-066' (longueur: 10)

Groupe PGC 45717:
  - PGC045717 | nom: 'ESO269-058' (longueur: 10) [GARDE]
  - PGC45717 | nom: 'ESO269-058' (longueur: 10)

Groupe PGC 45917:
  - PGC45917 | nom: 'ESO269-068' (longueur: 10) [GARDE]
  - PGC045917 | nom: 'NGC5011C' (longueur: 8)

Groupe PGC 47171:
  - PGC047171 | nom: 'ESO324-024' (longueur: 10) [GARDE]
  - PGC47171 | nom: 'ESO324-024' (longueur: 10)

Groupe PGC 48738:
  - PGC48738 | nom: 'ESO325-011' (longueur: 10) [GARDE]
  - PGC048738 | nom: 'ESO325-011' (longueur: 10)

Groupe PGC 49050:
  - PGC049050 | nom: 'ESO383-087' (longueur: 10) [GARDE]
  - PGC49050 | nom: 'ESO383-087' (longueur: 10)

Groupe PGC 46663:
  - PGC46663